## Chapter 7: data cleaning and preparation

In [1]:
import sys
import pandas as pd
import numpy as np

Durante el curso de análisis y modelado de datos, se dedica una cantidad significativa
de tiempo a la preparación de datos: carga, limpieza, transformación y reorganización.
A menudo se informa que tales tareas ocupan el 80% o más del tiempo de un analista.
A veces, la forma en que los datos se almacenan en archivos o bases de datos no están
en el formato correcto para una tarea en particular. Muchos invetigadores optan por
realizar un procesamiento _ad hoc_ de una forma a otra utilizando un lenguage de
programación de propósito general, como Python, Perl, R o Java, o herramientas de
procesamiento de texto Unix como `sed` o `awk`. Afortunadamente, Pandas, junto con las
características integradas de Python, le proporcionan un conjunto de herramientas de
alto nivel, flexibles y rápidas para permitirle manipular los datos en la forma correcta.

### Manejo de datos faltantes
Los datos faltantes ocurren comúnmente en muchas aplicaciones de análisis de datos.
Uno de los objetivos de Pandas es hacer que trabajar con datos faltantes sea lo más
indoloro posible. Por ejemplo, todas las estadísticas descriptivas sobre objetos
Pandas excluye los datos faltantes de forma predeterminada.

La forma en que los datos faltantes se representan en los objetos Pandas es algo
imperfecta, pero es suficiente para la mayoría del uso del mundo real. Para datos
con dtype `float64`, pandas utiliza el valor de punto flotante `NaN` (no es un número)
para representar los datos faltantes.

Llamamos a esto un valor centinela: cuando está presente, indica un valor faltante (o nulo):

In [2]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [3]:
# isna devuelve una serie booleana con True en valores nulos
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

En Pandas, hemos adaptado una convención utilizada en el lenguaje de programación R
refiréndonos a los datos faltantes como NA, que significa _no disponible_. En las
aplicaciones estadísticas, los datos de NA pueden ser datos que no existen o que existen,
pero no se observaron. Al limpiar los datos para el análisis, a menudos es importante
hacer análisis sobre los datos faltantes para identificar problemas de recopilación de
datos o posibles sesgos en los datos causados por los datos faltantes.

In [4]:
# el valor None también se trata como NA
string_data = pd.Series(["aardvark", np.nan, None, "aguacate"])
string_data

0    aardvark
1         NaN
2        None
3    aguacate
dtype: object

In [5]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [6]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [7]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

In [8]:
float_data.dropna() # evita los valores NA

0    1.0
1    2.0
dtype: float64

In [9]:
float_data.fillna("NULL") # sustituye valores faltantes por otro valor

0     1.0
1     2.0
2    NULL
dtype: object

In [10]:
float_data.notna() # True en los no faltantes

0     True
1     True
2    False
dtype: bool

### Filtrar datos faltantes
Hay algunas formas de filtrar los datos faltantes. Si bien siempre tiene la opción de
hacerlo a mano usando `pd.isna` e indexación booleana, `dropna` puede ser útil.
En una serie, devuelve la serie con solo los datos no nulos y los valores de índice:

In [11]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [12]:
# es lo mismo que hacer
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

Con los objetos DataFrame, hay diferentes formas de eliminar los datos faltantes. Es
posible que desee soltar filas o columnas que son todas NA, o solo aquellas filas o
columnas que contienen NA, `dropna` por defecto, suelta cualquier fila que contenga
un valor faltante:

In [13]:
data = pd.DataFrame([[1., 6.5, 3.],
                    [1., np.nan, np.nan],
                    [np.nan, np.nan, np.nan],
                    [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [14]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Pasando `how="all"` solo soltará filas que sean todas NA:

In [15]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Para soltar columnas `axis="columns"`:

In [16]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [17]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Supongamos que desea mantener solo filas que contengan como máximo un cierto número de
observaciones faltantes. Puede indicar esto con el argumento `thresh`:

In [18]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df

,0,1,2
0,1.323947,-0.840826,-0.491723
1,-1.470630,0.811929,-0.280962
2,-0.120286,-0.284286,1.663586
3,-0.881827,1.835930,-0.164956
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


In [19]:
df.iloc[:4, 1] = np.nan

In [20]:
df

,0,1,2
0,1.323947,NaN,-0.491723
1,-1.470630,NaN,-0.280962
2,-0.120286,NaN,1.663586
3,-0.881827,NaN,-0.164956
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


In [21]:
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,1.323947,NaN,NaN
1,-1.470630,NaN,NaN
2,-0.120286,NaN,1.663586
3,-0.881827,NaN,-0.164956
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


In [22]:
df.dropna()

,0,1,2
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


In [23]:
df.dropna(thresh=2)

,0,1,2
2,-0.120286,NaN,1.663586
3,-0.881827,NaN,-0.164956
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


### Rellenando datos faltentes
En lugar de filtrar los datos faltantes (y potencialmente descartar otros datos junto
con ellos), es posible que desee completar los faltantes de varias maneras. Para la
mayoría de los propósitos, el método `fillna` puede ser apropiado. Llamando a `fillna`
con una constante reemplaza los valores faltantes con ese valor:

In [24]:
df.fillna(0)

,0,1,2
0,1.323947,0.000000,0.000000
1,-1.470630,0.000000,0.000000
2,-0.120286,0.000000,1.663586
3,-0.881827,0.000000,-0.164956
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


Llamando `fillna` con un diccionario puede usar un valor diferentes para cada columna:

In [25]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.323947,0.500000,0.000000
1,-1.470630,0.500000,0.000000
2,-0.120286,0.500000,1.663586
3,-0.881827,0.500000,-0.164956
4,0.275722,-0.056232,0.082396
5,-0.242533,-0.364503,-0.567399
6,1.061393,-0.835134,0.945479


In [26]:
# cambiar las columnas ya definidas 
# df.columns = ["one", "two", "three"]
# df.index

Los mismo métodos de interpolación disponibles para la reindexación se pueden utiliza
con `fillna`:

In [27]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,-0.974453,-0.337049,0.977572
1,-0.848565,0.237909,0.802637
2,0.862631,NaN,0.877783
3,0.474070,NaN,1.437690
4,-0.977567,NaN,NaN
5,1.056302,NaN,NaN


In [28]:
df.fillna(method="ffill")

,0,1,2
0,-0.974453,-0.337049,0.977572
1,-0.848565,0.237909,0.802637
2,0.862631,0.237909,0.877783
3,0.474070,0.237909,1.437690
4,-0.977567,0.237909,1.437690
5,1.056302,0.237909,1.437690


In [29]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,-0.974453,-0.337049,0.977572
1,-0.848565,0.237909,0.802637
2,0.862631,0.237909,0.877783
3,0.474070,0.237909,1.437690
4,-0.977567,NaN,1.437690
5,1.056302,NaN,1.437690


Con `fillna` puede hacer muchaas otras coas, como la simple imputación de datos
utilizando las estadísticas medianas o medias:

In [30]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Transformación de datos
El filtrado, la limpieza y otras transformaciones son otra clase de operaciones
importantes.

### Eliminar duplicados
Las filas duplicadas se pueden encontrar en un DataFrame por cualquier número de razones:

In [31]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                    "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


El método DataFame `duplicated` devuelve una Serie booleana que indica si cada fila es
un duplicado o no:

In [32]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicated` devuelve un DataFrame con filas donde `duplicated` es `False`:

In [33]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Ambos métodos, por defecto, consideran todas las columnas; alternativamente, puede
especificar cualquier subconjunto de ellas para detectar duplicados. Supongamos que
teníamos una columna adicional de valores y quisíeramos filtrar duplicados basados
solo en la columna `"k1"`:

In [34]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [35]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` y `drop_duplicates` por defecto, mantienen la primera combinación de
valores. Pasando `keep="last"` devolverá el último:

In [36]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Tranformar datos usando una función o mapeo
Para muchos conjuntos de datos, es posible que desee realizar alguna transformación
basada en valores de una matriz, Serie o columna en un DataFrame.

In [37]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                             "pastrami", "corned beef", "bacon",
                             "pastrami", "honey ham", "nova lox"],
                    "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Supongamos que desea agregar una columna que indique el tipo de animal del que proviene
cada alimento. Escribamos un mapeo de cada tipo de carne distinto al tipo de animal:

In [38]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salom"
}

El método `map` en una serie acepta una función o un objeto similar a un diccionario
que contiene un mapeo para hacer la transformación de valores:

In [39]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salom


In [40]:
# usar una función
def get_animal(x):
    return meat_to_animal[x]


data["food"].map(get_animal)

0      pig
1      pig
2      pig
3      cow
4      cow
5      pig
6      cow
7      pig
8    salom
Name: food, dtype: object

In [41]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salom


Usando `map` es una forma conveniente de realizar transformaciones en elementos y otras
operaciones realacionadas con la limpieza de datos.

### Sustitución de valores
Rellenar los datos faltantes con el método `fillna` es un caso especial de reemplazo de
valores más general. `map` se puede utilizar para modificar un subconjunto de valores en
un objeto, pero `replace` proporciona una forma más simple y flexible de hacerlo:

In [42]:
data = pd.Series([1., -999., 2, -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Los valores `-999` pueden ser valores centienla para datos faltantes. Para reemplazarlos
con valores NA que Pandas entienda, podemos usar `replace` para producir una nueva serie:

In [43]:
data.replace(-999, np.nan, inplace=True)
data

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [44]:
# pasar una lista para varios valores
data.replace([-999, -1000], np.nan, inplace=True)
data

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [45]:
# pasar una lista de sustitución para cada valor
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [46]:
# pasar un diccionario
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [47]:
# pasar un diccionario. usar inplace=True para afectar al DataFrame
data.replace({-999: np.nan, -1000: 0}, inplace=True)
data

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

### Renombar los índices de eje
Al igual que los valores en una serie, las etiquetas de los ejes pueden transformar de
manera similar mediante una función o mapeo de alguna forma para producir objetos
nuevos etiquetados de manera diferente. También puede modificar los ejes en su lugar
sin crear una nueva estructura de datos:

In [48]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=["Ohio", "Colorado", "New York"],
                   columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


al igual que una serie, los índices del eje tienen un método `map`:

In [49]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Puede asignar el atributo a `index`, modificando el DataFrame en su lugar:

In [50]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Si desea crear una nueva versión transformada de un conjunto de datos sin modificar el
original, un método útil es `rename`:

In [51]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


En particular, `rename` se puede utilizar junto con un objeto similar a un diccionario,
proporcionando nuevos valores para un subconjunto de las etiquetas de eje:

In [52]:
data.rename(index={"Ohio": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` le ahorra la tarea de copiar el DataFrame manualmente y asignar nuevos valores
a sus atributos `index` y `columns`.

### Descretization y Binning
Los datos continuos a menudo se descretizan o separan en "bins" para su análisis.
Supongamos que tiene datos sobre un grupo de personas en un estudio, y desea agruparlos
en cubos discretos de edad:

In [53]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [54]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

El objeto que devuelve pandas es un objeto categórico especial. La salida que ve
describe los contenedores calculados por `pandas.cut`. Cada contenedor se identifica
mediante un tipo de valor de intervalo especial (único para pandas) que contiene el
límite y superior de cada contenedor:

In [55]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [56]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

Tenga en cuenta que `pd.value_counts(categories)` son los recuentos de basura para el
resultado de `pandas.cut`.

En la representación de cadena de un intervalo, un paréntesis significa que el lado
es abierto (exclusivo), mientras que el soporte cuadrado significa que es cerrado
(inclusive). Puede cambiar qué lado está cerrado al pasar `right=False`:

In [57]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

Puede anular el etiquetado de bin predeterminado basado en intervalos pasando una lista
o matriz a la opción `labels`:

In [58]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senio"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senio', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senio']

Si pasa un número entero de contenedores a `pandas.cut` en lugar de bordes de contenedor
explícitos, calculará contenedores de igual longitud en función de los valores mínimo
y máximo en los datos. Considere el caso de algunos datos distribuidos uniformemente
cortados en cuartos:

In [59]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.21, 0.4], (0.59, 0.78], (0.59, 0.78], (0.21, 0.4], (0.4, 0.59], ..., (0.21, 0.4], (0.026, 0.21], (0.59, 0.78], (0.026, 0.21], (0.026, 0.21]]
Length: 20
Categories (4, interval[float64, right]): [(0.026, 0.21] < (0.21, 0.4] < (0.4, 0.59] < (0.59, 0.78]]

La opción `precision=2` limita la precisión decimal a dos dígitos.

Una función estrechamente relacionada, `pandas.qcut`, contiene los datos basados en
cuantiles de muetras. Dependiendo de la distribución de los datos. Desde `pd.qcut`
utiliza cuantiles de muestra en su lugar, obtendrá contenedores de aproximadamente el
mismo tamaño:

In [60]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-3.2199999999999998, -0.69], (-0.69, -0.029], (-0.029, 0.69], (-3.2199999999999998, -0.69], (-3.2199999999999998, -0.69], ..., (0.69, 2.95], (0.69, 2.95], (-3.2199999999999998, -0.69], (-0.69, -0.029], (-0.029, 0.69]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.2199999999999998, -0.69] < (-0.69, -0.029] < (-0.029, 0.69] < (0.69, 2.95]]

In [61]:
pd.value_counts(quartiles)

(-3.2199999999999998, -0.69]    250
(-0.69, -0.029]                 250
(-0.029, 0.69]                  250
(0.69, 2.95]                    250
dtype: int64

Similar a `pandas.cut`, puede pasar sus propios cuantiles (números entre 0 y 1):

In [62]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.215, -1.312]     100
(-1.312, -0.0287]    400
(-0.0287, 1.354]     400
(1.354, 2.951]       100
dtype: int64

### Detectar y filtrar outliers
Filtrar y transformar valores atípicos es en gran medida una cuestion de aplicar
operaciones de matriz:

In [63]:
data = pd.DataFrame(np.random.standard_normal((100, 4)))
data.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,0.145968,-0.119231,0.179414,0.059194
std,1.067935,0.917994,1.119710,1.008675
min,-2.266735,-2.310433,-2.467171,-2.134278
25%,-0.501237,-0.614422,-0.622558,-0.728923
50%,0.222664,-0.006860,0.191672,0.112354
75%,0.899744,0.469368,1.061315,0.678552
max,3.339686,1.512681,2.352610,2.295397


Supongamos que desea encontrar valores en una de las columnas que excede 3 en valor
absoluto:

In [64]:
col = data[2]
col[col.abs() > 3]

Series([], Name: 2, dtype: float64)

Seleccionar todas las filas que tengan un valor superior a 3, puede usar `any` en
un DataFrame booleano:

In [65]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
40,3.339686,0.793504,0.736351,-1.158407


Los paréntesis alrededor de `data.abs() > 3` son necesarios para llamar a `any` sobre
la operación de comparación.

Los valores se pueden establecer en función de estos críterios. Aquí hay valores de
código para limitar fuera del intervalo 3 a 3:

In [66]:
data[data.abs() >3] = np.sign(data) * 3

In [67]:
data.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,0.142571,-0.119231,0.179414,0.059194
std,1.058170,0.917994,1.119710,1.008675
min,-2.266735,-2.310433,-2.467171,-2.134278
25%,-0.501237,-0.614422,-0.622558,-0.728923
50%,0.222664,-0.006860,0.191672,0.112354
75%,0.899744,0.469368,1.061315,0.678552
max,3.000000,1.512681,2.352610,2.295397


La declaración `np.sign(data)` produce valores de 1-1 en función de si los valores en
`data` son positivos o negativos:

In [68]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,1.0,1.0,1.0
2,-1.0,1.0,-1.0,-1.0
3,1.0,1.0,-1.0,-1.0
4,1.0,1.0,-1.0,1.0


### Permutación y muestreo aleatorio
Permutar (reordenar aleatoriamente) una Serie o filas en un DataFrame es posible usando
la función `numpy.random.permutation`. Llamando `permutation` con la longitud del eje
que desea permutar produce una matriz de enteros que indica el nuevo orden:

In [69]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [70]:
sampler = np.random.permutation(5)
sampler

array([0, 1, 2, 3, 4])

In [71]:
df.take(sampler)

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [72]:
df.iloc[sampler]

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [73]:
# seleccionar una permutación por columnas take con axis="columns"
column_sampler = np.random.permutation(7)
column_sampler

array([3, 2, 1, 5, 6, 4, 0])

In [74]:
df.take(column_sampler, axis="columns")

,3,2,1,5,6,4,0
0,3,2,1,5,6,4,0
1,10,9,8,12,13,11,7
2,17,16,15,19,20,18,14
3,24,23,22,26,27,25,21
4,31,30,29,33,34,32,28


Para seleccionar un subconjunto aleatorio sin reemplazo, puede usar el método
`sample` en Series y DataFrame:

In [75]:
df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13


Para generar una muestra sin reemplazo (permitir opciones repetidas):

In [76]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

2   -1
4    4
4    4
3    6
2   -1
2   -1
3    6
1    7
0    5
3    6
dtype: int64

### Indicador de computación/variables ficticias
Otro tipo de transformación para aplicaciones de modelado estadístico o aprendizaje
automático es convertir una variable categórica en un _dummy_ o indicador de matriz.
Si una columna en un DataFrame tiene `k` valores distintos, derivaría una matriz o
DataFrame con `k` columnas que contiene todos los 1's o 0's. Pandas tiene una función
`pandas.get_dummies` para hacer esto, aunque también podría idear uno usted mismo.

In [77]:
df = pd.DataFrame({"key": list("bbacab"),
                  "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [78]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


En algunos casos, es posible que desee agregar un prefijo a las columnas en el
indicador DataFrame, que luego se puede fusionar con los otros datos.

In [79]:
dummies = pd.get_dummies(df["key"], dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,a,b,c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


Si una fila en un DataFrame pertenece a múltiples categorías, tenemos que usar un
enfoque diferente para crear las variables ficticias.

In [80]:
mnames = ["movie_id", "title", "genres"]

movies = pd.read_table("../files/datasets/movielens/movies.dat", sep="::",
                      header=None, names=mnames, engine="python")
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Pandas ha implementado un método especial de la serie `str.get_dummies` que maneja
este escenario de membresía de grupos múltiples codificada como una cadena delimitada:

In [81]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


Entonces, como antes, puede combinar esto con `movies` mientras agrega un `"genre_"`
a los nombres de columnas en el DataFrame `dummies` con el método `add_prefix`:

In [82]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Una receta útil para aplicaciones estadísticas es combinar `pandas.get_dummies` con una
función de descretización como `pandas.cut`:

In [83]:
np.random.seed(12345)
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [84]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


### Tipos de datos de extensión
Pandas se contruyó originalmente sobre las capacidades presentes en NumPy, una
biblioteca de computación de matriz utilizada principalmente para trabajar con datos
numéricos. Muchos conceptos de pandas, como los datos faltantes, se implementaron
utilizando lo que estaba disponible en NumPy mientras intentaba maximizae la
compatibilidad entre las bibliotecas que usaba Numpy y Pandas.

La construcción de Numpy condujo a una serie de deficiencias, tales como:
- El manejo de datos faltante para algunos tipos de datos numéricos, como enteros y booleanos, estaba incompleto. Como resultado, cuando se introdujeron datos faltantes en dichos datos, Pandas convertía el tipo de datos en `float64` y utilizando `np.nan` para representar valores nulos. Esto tuvo efectos compuestos al introducir problemas sutiles en muchos algoritmos de Pandas.
- Los conjuntos de datos con muchos datos de cadena eran computacionalmente caros y usaban mucha memoria.
- Algunos tipos de datos, como intervalos de timepo, las marcas de timpo y las marcas de tiempo de zonas horarias, no se pueden admitir de manera eficiente sin usar matrices computacionales caras de objetos Python.

Más recientemente, Pandas ha desarrollado un tipo de extensión de sistema que permite
agregar nuevos tipos de datos incluso si Numpy no los admite de forma nativa. Estos
nuevos tipos de datos se pueden tratar como de primera clase junto con los datos
procedentes de las matrices NumPy.

In [85]:
s = pd.Series([1, 2, 3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [86]:
s.dtype

dtype('float64')

Por razones de compatibilidad con versiones anteriores, Series utiliza el comportamiento
heredado de usar un tipo de dato `float64` y `np.nan` para el valor perdido.

In [87]:
# usar pandas.Int64Dtype
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

In [88]:
s.dtype

Int64Dtype()

In [89]:
s.isna()

0    False
1    False
2    False
3     True
dtype: bool

La salida `<NA>` indica que falta un valor para una matriz de tipo de extensión. Esto
usa el valor especial `pandas.NA`:

In [90]:
s[3]

<NA>

In [91]:
s[3] is pd.NA

True

In [92]:
# usar In64 en lugar de pd.In64Dtype()
s = pd.Series([1, 2, 3, None], dtype="Int64")
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

tipo de extensión especializado para datos de cadenas que no utilizan matrices de
objetos NumPy:

In [93]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

Las matrices de cadena generalmente usan menos memoria y con frecuencia son
computacionalmente más eficientes para realizar operaciones en granfes conjuntos de
datos.

Los tipos de extensión se pueden pasar a la serie con el método `astype`, que permite
convertir fácilmente como parte de su proceso de limpieza de datos:

In [94]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                  "B": ["one", "two", "three", None],
                  "C": [False, None, False, True]})
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [95]:
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


In [96]:
df.isna()

,A,B,C
0,False,False,False
1,False,False,True
2,True,False,False
3,False,True,False


### Métodos de objetos de cadena integrados de Python
En muchas aplicaciones de scripting y cadenas, los métodos de cadena incorporados son
suficientes. Como ejemplo, una cadena separada por comas se puede romper en pedazos con
`split`:

In [97]:
val = "a,b, guido"
val.split(",")

['a', 'b', ' guido']

`split` a menudo se combina con `strip` para recortar espacios en blanco (incluidos
saltos de línea):

In [98]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

Estas subcadenas podrían concatenarse junto con un delimitador de dos puntos:

In [99]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

In [100]:
"::".join(pieces)

'a::b::guido'

### Expresiones regulares
Las _expresionres regulares_ proporcionan una forma flexible de buscar o hacer coincidir
(a menudo más compleja) patrones de cadena en el texto. Una sola expresión, comúnmente
llamada _regex_, es una cadena formada de acuerdo con el lenguaje de expresión regular.
Python integra el módulo `re`, que es responsable de aplicar expresiones regulares a las
cadenas.

La función `re` del módulo se divide en tres categorías: coincidencias de patrones,
sustitución y división. Naturalmente, todos estos están relacionados; un regex describe
un patrón para ubicar en el texto, que luego se puede usar para muchos propósitos.
Supongamos que queríamos dividir una cadena con un número variable de caracteres de
espacios en blanco (tabulaciones, espacios y saltos de línea).

El regex que describe uno o más caracteres de espacios en blanco es `\s+`:

In [101]:
import re

text = "foo    bar\t baz  \tqux"

re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

Cuando llama a `re.split(r"\s+", text)`, la expresión regular es primero compilada, y
luego su método `split`se llama en el texto pasado. Puede compilar el regex con
`re.compile`, formando un objeto regex reutilizable:

In [102]:
regex = re.compile(r"\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

Si, en cambio, desea obtener una lista de todos los patrones que coinciden con el
regex, puede usar el método `findall`:

In [103]:
regex.findall(text)

['    ', '\t ', '  \t']

Crear un objeto regex con `re.compile` es muy recomendable si tiene la intención de
aplicar la misma expresión a muchas cadenas; hacerlo ahorrará ciclo de CPU.

`match` y `search` están estrechamente relacionados con `findall`. Mientras `findall`
devuelve todas las coincidencias en una cadena, `search`devuelve solo la primera
coincidencia. Más rigidamente, `match` solo coincide al comienzo de la cadena. Como
ejemplo, consideremos un bloque de texto y una expresión regular capaz de identificar la
mayoría de las direcciones de correo electrónico:

In [104]:
text = """
Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

regex = re.compile(pattern, flags=re.IGNORECASE)

Usando `findall` en el texto se produce una lista de las direcciones de correo:

In [105]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` devuelve un objeto de coincidencia especial para la primera dirección de
correo en el texto. Para el regex anterior, el objeto `match` solo puede decirnos la
posición inicial y final del patrón en la cadena:

In [106]:
m = regex.search(text)
m

<re.Match object; span=(6, 21), match='dave@google.com'>

In [107]:
text[m.start():m.end()]

'dave@google.com'

`regex.match` devuelve `None`, ya que coincidirá solo si el patrón ocurre al comienzo de
la cadena:

In [108]:
print(regex.match(text))

None


Relacionadamente, `sub` devolverá una nueva cadena con ocurrencias del patrón
reemplazadas por una nueva cadena:

In [109]:
print(regex.sub("REDACTED", text))


Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Supongamos que desea encontrar direciones de correo y al mismo tiempo segmentar cada
dirección en sus tres componentes: nombre de usuario, nombre de dominio y sufijo de
dominio. Para hacer esto, colocar paréntesis alrededor de las partes del patrón para segmentar:

In [110]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

In [111]:
m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

`findall` devuelve una lista de tuplas cuando el patrón tiene grupos:

In [112]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]